In [1]:
# Note: Remove 'Dropout' layer if you use this for iOS because there are no support for Dropout on iOS and it will trigger the error message like this:
# # Invalid argument: No OpKernel was registered to support Op 'RandomUniform' with these attrs.  Registered devices: [CPU], Registered kernels:
# #   <no registered kernels>
# # 	 [[Node: dropout/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=0, seed2=0](dropout/Shape)]]
# # Invalid argument: Session was not created with a graph before Run()!
# Read here for more detail: http://www.mattrajca.com/2016/11/25/getting-started-with-deep-mnist-and-tensorflow-on-ios.html
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [3]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [4]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

In [5]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1], padding='SAME')

In [6]:
if __name__ == '__main__':
    mnist = input_data.read_data_sets('data', one_hot=True)
    x = tf.placeholder("float", shape=[None, 784], name="x")
    y_ = tf.placeholder("float", shape=[None, 10])
    sess = tf.InteractiveSession()

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [7]:
    x_image = tf.reshape(x, [-1,28,28,1])

    W_conv1 = weight_variable([3, 3, 1, 32])
    b_conv1 = bias_variable([32])
    W_conv2 = weight_variable([3, 3, 32, 32])
    b_conv2 = bias_variable([32])
    W_fc1 = weight_variable([12*12*32, 128])
    b_fc1 = bias_variable([128])
    W_fc2 = weight_variable([128, 10])
    b_fc2 = bias_variable([10])

In [8]:
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

In [9]:
    h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)

In [10]:
    h_pool = max_pool_2x2(h_conv2)

In [11]:
#     keep_prob1 = tf.placeholder("float")

In [12]:
#     h_drop1 = tf.nn.dropout(h_pool, keep_prob1)

In [16]:
    h_flat = tf.reshape(h_pool, [-1, 12*12*32])
    #     h_flat = tf.reshape(h_drop1, [-1, 12*12*32])

In [17]:
    h_fc1 = tf.nn.relu(tf.matmul(h_flat, W_fc1) + b_fc1)

In [18]:
#     keep_prob2 = tf.placeholder("float")

In [19]:
#     h_drop2 = tf.nn.dropout(h_fc1, keep_prob2)

In [20]:
    y_conv = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2, name="softmax")
#     y_conv = tf.nn.softmax(tf.matmul(h_drop2, W_fc2) + b_fc2, name="softmax")

In [21]:
    cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    init = tf.initialize_all_variables()
    saver = tf.train.Saver()
    sess.run(init)

In [22]:
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i%100 == 0:
#             train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob1: 1.0, keep_prob2: 1.0})
            train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print("step %d, training accuracy %g"%(i, train_accuracy))
#         train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob1: 0.25, keep_prob2: 0.5})
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

step 0, training accuracy 0.08
step 100, training accuracy 0.66
step 200, training accuracy 0.88
step 300, training accuracy 0.8
step 400, training accuracy 0.98
step 500, training accuracy 0.86
step 600, training accuracy 0.96
step 700, training accuracy 0.9
step 800, training accuracy 0.86
step 900, training accuracy 0.94
step 1000, training accuracy 0.9
step 1100, training accuracy 0.96
step 1200, training accuracy 0.98
step 1300, training accuracy 0.96
step 1400, training accuracy 0.96
step 1500, training accuracy 0.88
step 1600, training accuracy 0.96
step 1700, training accuracy 0.94
step 1800, training accuracy 0.9
step 1900, training accuracy 0.98
step 2000, training accuracy 0.96
step 2100, training accuracy 0.94
step 2200, training accuracy 0.96
step 2300, training accuracy 0.96
step 2400, training accuracy 0.92
step 2500, training accuracy 1
step 2600, training accuracy 0.94
step 2700, training accuracy 0.94
step 2800, training accuracy 0.96
step 2900, training accuracy 1
st

In [23]:
    saver.save(sess, "model.ckpt")
    tf.train.write_graph(sess.graph_def, '', 'graph.pb')

In [24]:
    print("test accuracy %g"%accuracy.eval(feed_dict={
#         x: mnist.test.images, y_: mnist.test.labels, keep_prob1: 1.0, keep_prob2: 1.0}))
            x: mnist.test.images, y_: mnist.test.labels}))

test accuracy 0.9872
